In [163]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import time
import pickle
import re

In [152]:
# Setting pandas to display max columns
pd.set_option('display.max_columns', None)
# Setting pandas to display max rows
pd.set_option('display.max_rows', None)

In [3]:
nfl_small2 = pd.read_csv('nfl_small_cleaned_plus_weather.csv').drop(columns=['Unnamed: 0']).sort_values(by=['game_id','play_id'])
nfl_small2.shape

(87272, 67)

In [4]:
nfl_small2.columns

Index(['index', 'play_id', 'game_id', 'game_date', 'time',
       'quarter_seconds_remaining', 'half_seconds_remaining',
       'game_seconds_remaining', 'game_half', 'quarter_end', 'qtr',
       'home_team', 'away_team', 'posteam', 'posteam_type', 'defteam',
       'side_of_field', 'yardline_100', 'drive', 'sp', 'down', 'goal_to_go',
       'yrdln', 'ydstogo', 'ydsnet', 'desc', 'play_type', 'yards_gained',
       'pass_length', 'air_yards', 'yards_after_catch', 'field_goal_result',
       'kick_distance', 'extra_point_result', 'two_point_conv_result',
       'td_team', 'total_home_score', 'total_away_score', 'posteam_score',
       'defteam_score', 'sack', 'touchdown', 'pass_touchdown',
       'rush_touchdown', 'return_touchdown', 'extra_point_attempt',
       'two_point_attempt', 'field_goal_attempt', 'fumble', 'complete_pass',
       'Start_Time', 'diff', 'Estimated_Time', 'Estimated_Hour',
       'Air Pressure (hPa)', 'City', 'Dewpoint (°C)', 'Field', 'Humidity (%)',
       'Precip

In [5]:
nfl_small2.sample(5)

,index,play_id,game_id,game_date,time,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,qtr,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,drive,sp,down,goal_to_go,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,pass_length,air_yards,yards_after_catch,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,td_team,total_home_score,total_away_score,posteam_score,defteam_score,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,fumble,complete_pass,Start_Time,diff,Estimated_Time,Estimated_Hour,Air Pressure (hPa),City,Dewpoint (°C),Field,Humidity (%),Precipitation (mm),Roof,Team Abbreviation,Temperature (°C),Time (EST),Time (GMT),Wind Direction (deg),Wind Speed (km/h)
39946,402045,3642,2017122301,2017-12-23 00:00:00,02:35,155.0,155.0,155.0,Half2,0,4,GB,MIN,GB,home,MIN,GB,76.0,23,0,2.0,0.0,GB 24,20,9,(2:35) (Shotgun) B.Hundley pass deep middle to...,pass,19.0,deep,18.0,1.0,NaN,NaN,NaN,NaN,NaN,0,16,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2017-12-23 20:30:00,0 days 02:52:15.000000000,2017-12-23 23:22:15,2017-12-23 23:00:00,NaN,Green Bay,-14.0,Desso GrassMaster,53.0,NaN,Open,GB,-6.0,2017-12-23 23:00:00,2017-12-24 03:00:00,270.0,16.6
34166,396265,1354,2017121000,2017-12-10 00:00:00,01:54,114.0,114.0,1914.0,Half1,0,2,BUF,IND,IND,away,BUF,IND,55.0,8,0,NaN,0.0,IND 45,0,-6,Timeout #2 by BUF at 01:54.,no_play,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-12-10 13:00:00,0 days 01:24:18.000000000,2017-12-10 14:24:18,2017-12-10 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79071,441170,3398,2018120200,2018-12-02 00:00:00,14:13:00,853.0,853.0,853.0,Half2,0,4,ATL,BAL,ATL,home,BAL,ATL,69.0,16,0,4.0,0.0,ATL 31,4,6,"(14:13) M.Bosher punts 53 yards to BAL 16, Cen...",punt,0.0,NaN,NaN,NaN,NaN,53.0,NaN,NaN,NaN,9,16,9.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-12-02 16:25:00,0 days 02:17:21.000000000,2018-12-02 18:42:21,2018-12-02 19:00:00,1008.5,Atlanta,7.7,FieldTurf Revolution 360,50.0,0.0,Retractable,ATL,18.3,2018-12-02 19:00:00,2018-12-02 23:00:00,230.0,9.4
19883,381279,376,2017102905,2017-10-29 00:00:00,09:47,587.0,1487.0,3287.0,Half1,0,1,NYJ,ATL,ATL,away,NYJ,ATL,65.0,2,0,1.0,0.0,ATL 35,10,9,"(9:47) M.Ryan FUMBLES (Aborted) at ATL 35, REC...",run,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2017-10-29 13:00:00,0 days 00:15:39.000000000,2017-10-29 13:15:39,2017-10-29 13:00:00,NaN,East Rutherford,16.7,UBU Sports Speed Series S5-M Synthetic Turf,97.0,0.5,Open,NYJ,17.2,2017-10-29 13:00:00,2017-10-29 17:00:00,90.0,7.6
50141,412431,3007,2018091609,2018-09-16 00:00:00,10:29:00,629.0,629.0,629.0,Half2,0,4,LA,ARI,LA,home,ARI,LA,52.0,17,0,2.0,0.0,LA 48,4,50,(10:29) J.Goff pass incomplete short right to ...,pass,0.0,short,2.0,NaN,NaN,NaN,NaN,NaN,NaN,27,0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-09-16 16:25:00,0 days 02:28:33.000000000,2018-09-16 18:53:33,2018-09-16 19:00:00,1011.5,Los Angeles,16.2,Grass,62.0,0.0,Open,LA,23.9,2018-09-16 19:00:00,2018-09-16 23:00:00,250.0,18.4


In [6]:
nfl_small2[['game_date', 'Time (EST)', 'Time (GMT)', 'Estimated_Time', 'Start_Time', 'Estimated_Hour']].dtypes

game_date         object
Time (EST)        object
Time (GMT)        object
Estimated_Time    object
Start_Time        object
Estimated_Hour    object
dtype: object

In [7]:
time_cols = ['game_date', 'Time (EST)', 'Time (GMT)', 'Estimated_Time', 'Start_Time', 'Estimated_Hour']

for col in time_cols:
    nfl_small2[col] = pd.to_datetime(nfl_small2[col])
    
nfl_small2[['game_date', 'Time (EST)', 'Time (GMT)', 'Estimated_Time', 'Start_Time', 'Estimated_Hour']].dtypes

game_date         datetime64[ns]
Time (EST)        datetime64[ns]
Time (GMT)        datetime64[ns]
Estimated_Time    datetime64[ns]
Start_Time        datetime64[ns]
Estimated_Hour    datetime64[ns]
dtype: object

In [10]:
nfl_small2.sample(10)

,index,play_id,game_id,game_date,time,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,qtr,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,drive,sp,down,goal_to_go,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,pass_length,air_yards,yards_after_catch,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,td_team,total_home_score,total_away_score,posteam_score,defteam_score,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,fumble,complete_pass,Start_Time,diff,Estimated_Time,Estimated_Hour,Air Pressure (hPa),City,Dewpoint (°C),Field,Humidity (%),Precipitation (mm),Roof,Team Abbreviation,Temperature (°C),Time (EST),Time (GMT),Wind Direction (deg),Wind Speed (km/h)
38760,400859,156,2017121708,2017-12-17,13:36,816.0,1716.0,3516.0,Half1,0,1,SEA,LA,LA,away,SEA,SEA,40.0,2,0,1.0,0.0,SEA 40,10,14,(13:36) T.Gurley up the middle to SEA 26 for 1...,run,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-12-17 16:05:00,0 days 00:04:12.000000000,2017-12-17 16:09:12,2017-12-17 16:00:00,1019.0,Seattle,6.7,FieldTurf Revolution 360,86.0,0.0,Open,SEA,8.9,2017-12-17 16:00:00,2017-12-17 20:00:00,190.0,29.5
42541,407054,2584,2017123100,2017-12-31,06:35,395.0,1295.0,1295.0,Half2,0,3,ATL,CAR,ATL,home,CAR,CAR,12.0,13,1,4.0,0.0,CAR 12,10,68,"(6:35) M.Bryant 30 yard field goal is GOOD, Ce...",field_goal,0.0,NaN,NaN,NaN,made,30.0,NaN,NaN,NaN,10,7,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2017-12-31 16:25:00,0 days 01:55:15.000000000,2017-12-31 18:20:15,2017-12-31 18:00:00,1022.7,Atlanta,-3.9,FieldTurf Revolution 360,64.0,0.0,Retractable,ATL,2.2,2017-12-31 18:00:00,2017-12-31 22:00:00,330.0,20.5
79905,441834,1877,2018120205,2018-12-02,02:13:00,133.0,133.0,1933.0,Half1,0,2,JAX,IND,JAX,home,IND,IND,12.0,9,1,4.0,0.0,IND 12,10,48,"(2:13) J.Lambo 30 yard field goal is GOOD, Cen...",field_goal,0.0,NaN,NaN,NaN,made,30.0,NaN,NaN,NaN,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2018-12-02 16:25:00,0 days 01:23:21.000000000,2018-12-02 17:48:21,2018-12-02 18:00:00,1011.3,Jacksonville,21.0,Bermuda grass,69.0,0.0,Open,JAX,27.2,2018-12-02 18:00:00,2018-12-02 22:00:00,210.0,18.4
83144,444311,1708,2018120907,2018-12-09,04:18:00,258.0,258.0,2058.0,Half1,0,2,TB,NO,NO,away,TB,NO,66.0,8,0,1.0,0.0,NO 34,10,17,"(4:18) PENALTY on TB-V.Vea, Neutral Zone Infra...",no_play,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,3,3.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-12-09 16:25:00,0 days 01:17:06.000000000,2018-12-09 17:42:06,2018-12-09 18:00:00,1013.2,Tampa,17.8,Bermuda grass,84.0,0.0,Open,TB,20.6,2018-12-09 18:00:00,2018-12-09 22:00:00,280.0,13.0
26506,388605,1701,2017111902,2017-11-19,00:26,26.0,26.0,1826.0,Half1,0,2,GB,BAL,BAL,away,GB,GB,35.0,10,0,3.0,0.0,GB 35,15,9,(:26) (Shotgun) J.Flacco pass short middle to ...,pass,14.0,short,8.0,6.0,NaN,NaN,NaN,NaN,NaN,0,3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2017-11-19 13:00:00,0 days 01:28:42.000000000,2017-11-19 14:28:42,2017-11-19 14:00:00,NaN,Green Bay,-4.1,Desso GrassMaster,42.0,NaN,Open,GB,8.0,2017-11-19 14:00:00,2017-11-19 18:00:00,240.0,22.3
60486,422395,1224,2018101405,2018-10-14,10:44:00,644.0,644.0,2444.0,Half1,0,2,MIN,ARI,MIN,home,ARI,MIN,96.0,10,0,3.0,0.0,MIN 4,7,7,"(10:44) (Shotgun) PENALTY on MIN-B.O'Neill, Fa...",no_play,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,3,10.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-10-14 16:25:00,0 days 00:57:48.000000000,2018-10-14 17:22:48,2018-10-14 17:00:00,1020.6,Minneapolis,-1.1,UBU Speed Series S5-M Synthetic Turf,73.0,0.0,Fixed,MIN,3.3,2018-10-14 17:00:00,2018-10-14 21:00:00,300.0,16.6
60164,422956,2194,2018101403,2018-10-14,10:12:00,612.0,1512.0,1512.0,Half2,0,3,HOU,BUF,HOU,home,BUF,HOU,75.0,13,0,1.0,0.0,HOU 25,10,23,(10:12) (Shotgun) D.Watson sacked at HOU 23 fo...,pass,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,3,10.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [9]:
import plotly.graph_objects as go

In [81]:
nfl_small2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87272 entries, 0 to 87271
Data columns (total 67 columns):
index                        87272 non-null int64
play_id                      87272 non-null int64
game_id                      87272 non-null int64
game_date                    87272 non-null datetime64[ns]
time                         87272 non-null object
quarter_seconds_remaining    87272 non-null float64
half_seconds_remaining       87272 non-null float64
game_seconds_remaining       87272 non-null float64
game_half                    87272 non-null object
quarter_end                  87272 non-null int64
qtr                          87272 non-null int64
home_team                    87272 non-null object
away_team                    87272 non-null object
posteam                      87272 non-null object
posteam_type                 87272 non-null object
defteam                      87272 non-null object
side_of_field                87127 non-null object
yardline_100      

In [199]:
def my_aggfunc(x):
    values = {
        'pass_home':x[x['posteam_type']=='home']['play_type'].eq('pass').sum(),
        'run_home':x[x['posteam_type']=='home']['play_type'].eq('run').sum(),
        'pass_away':x[x['posteam_type']=='away']['play_type'].eq('pass').sum(),
        'run_away':x[x['posteam_type']=='away']['play_type'].eq('run').sum()
    }
    
    return pd.Series(values)
new_df = pd.DataFrame()
new_df = nfl_small2.groupby(['game_id']).apply(my_aggfunc)
new_df['home_team'] = nfl_small2.groupby('game_id')['home_team'].first()
new_df['away_team'] = nfl_small2.groupby('game_id')['away_team'].first()
new_df['home_score'] = nfl_small2.groupby('game_id')['total_home_score'].last()
new_df['away_score'] = nfl_small2.groupby('game_id')['total_away_score'].last()

In [202]:
new_df['Temperature (°C)'] = nfl_small2.groupby('game_id')['Temperature (°C)'].mean()
new_df['Precipitation (mm)'] = nfl_small2.groupby('game_id')['Precipitation (mm)'].mean()
new_df['Dewpoint (°C)'] = nfl_small2.groupby('game_id')['Dewpoint (°C)'].mean()
new_df['Humidity (%)'] = nfl_small2.groupby('game_id')['Humidity (%)'].mean()
new_df['Air Pressure (hPa)'] = nfl_small2.groupby('game_id')['Air Pressure (hPa)'].mean()
new_df['Wind Speed (km/h)'] = nfl_small2.groupby('game_id')['Wind Speed (km/h)'].mean()
new_df['Roof'] = nfl_small2.groupby('game_id')['Roof'].first()

In [203]:
new_df.head()

,pass_home,run_home,pass_away,run_away,home_team,away_team,home_score,away_score,Temperature (°C),Precipitation (mm),Dewpoint (°C),Humidity (%),Air Pressure (hPa),Wind Speed (km/h),Roof
game_id,,,,,,,,,,,,,,,
2017090700,39,34,38,24,NE,KC,27,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017091000,29,39,41,14,BUF,NYJ,21,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017091001,44,19,32,21,CHI,ATL,17,23,20.808589,0.0,6.574847,39.680982,1029.621472,16.600000,Open
2017091002,36,22,18,42,CIN,BAL,0,20,21.985207,0.0,8.540237,42.331361,1027.613018,20.918935,Open
2017091003,37,26,37,14,CLE,PIT,18,20,21.234146,0.0,8.131098,43.085366,1031.120732,12.983537,Open


In [204]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 2017090700 to 2018121700
Data columns (total 15 columns):
pass_home             480 non-null int64
run_home              480 non-null int64
pass_away             480 non-null int64
run_away              480 non-null int64
home_team             480 non-null object
away_team             480 non-null object
home_score            480 non-null int64
away_score            480 non-null int64
Temperature (°C)      461 non-null float64
Precipitation (mm)    418 non-null float64
Dewpoint (°C)         461 non-null float64
Humidity (%)          461 non-null float64
Air Pressure (hPa)    446 non-null float64
Wind Speed (km/h)     447 non-null float64
Roof                  462 non-null object
dtypes: float64(6), int64(6), object(3)
memory usage: 60.0+ KB


In [205]:
# null_vals = pd.isnull(new_df)
# xd = new_df[null_vals]

In [177]:
cleaned_df = new_df.dropna()

In [178]:
cleaned_df

,pass,run,home_team,away_team,Temperature (°C),Precipitation (mm),Dewpoint (°C),Humidity (%),Air Pressure (hPa),Wind Speed (km/h),Roof
game_id,,,,,,,,,,,
2017091001,76,40,CHI,ATL,20.808589,0.000000,6.574847,39.680982,1029.621472,16.600000,Open
2017091002,54,64,CIN,BAL,21.985207,0.000000,8.540237,42.331361,1027.613018,20.918935,Open
2017091003,74,40,CLE,PIT,21.234146,0.000000,8.131098,43.085366,1031.120732,12.983537,Open
2017091005,67,60,HOU,JAX,29.155191,0.000000,15.519126,43.661202,1018.127322,17.206557,Retractable
2017091007,76,49,TEN,OAK,25.494767,0.000000,12.230233,43.837209,1023.488372,13.795349,Open
2017091008,84,40,WAS,PHI,22.542105,0.000000,11.670760,50.251462,1028.647953,14.147368,Open
2017091009,56,52,LA,IND,25.513580,0.000000,16.279012,57.240741,1011.106173,20.079630,Open
2017091011,63,50,SF,CAR,33.611656,0.000000,15.139264,33.269939,1012.221472,14.711656,Open
2017091012,80,42,DAL,NYG,24.744970,0.000000,12.360947,46.230769,1019.509467,8.185799,Retractable
